In [1]:
pip install -U bitsandbytes>=0.46.1

In [2]:
# Clear GPU memory (run this cell to start fresh)
import torch
import gc

# Clear Python garbage collector
gc.collect()

# Clear PyTorch CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU cache cleared")
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
else:
    print("No CUDA device available")

GPU cache cleared
GPU memory allocated: 0.00 GB
GPU memory reserved: 0.00 GB


In [3]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Detect if running in Google Colab
IN_COLAB = False
cache_dir = None

try:
    import google.colab  # type: ignore
    IN_COLAB = True
    print("Running in Google Colab")

    # Mount Google Drive for persistent caching
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')

    # Set cache directory to Google Drive
    cache_dir = "/content/drive/MyDrive/huggingface_cache"
    os.makedirs(cache_dir, exist_ok=True)
    print(f"Cache directory: {cache_dir}")
except ImportError:
    print("Running locally")

# Configure 8-bit quantization to reduce memory usage
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

# Load model with quantization and automatic device mapping
model = AutoModelForImageTextToText.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    quantization_config=quantization_config,
    device_map="auto",
    cache_dir=cache_dir
)

processor = AutoProcessor.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    cache_dir=cache_dir
)

Using device: cuda
Running in Google Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cache directory: /content/drive/MyDrive/huggingface_cache


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:760: UserWarning: Not enough free disk space to download the file. The expected file size is: 9996.33 MB. The target location /content/drive/MyDrive/huggingface_cache/models--Salesforce--blip2-opt-2.7b/blobs only has 5972.11 MB free disk space.
  warnings.warn(


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
import zipfile
import os
import shutil
from PIL import Image
from pathlib import Path

# Define paths
zip_file_path = "uic.zip"
extract_to_path = "data/uic"
dataset_base = "data/uic/UIC(underwater image captioning dataset)"
image_dir = os.path.join(dataset_base, "uic_224x224_image")
captions_file = os.path.join(dataset_base, "UIC-captions.txt")

# Delete existing data/uic directory if it exists
if os.path.exists(extract_to_path):
    print(f"Removing existing {extract_to_path}...")
    shutil.rmtree(extract_to_path)

# Extract zip file if it exists
if os.path.exists(zip_file_path):
    os.makedirs(extract_to_path, exist_ok=True)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
    print(f"Successfully extracted {zip_file_path} to {extract_to_path}")

# Load captions from UIC-captions.txt
def load_captions(captions_path):
    """Parse UIC captions file and return a dictionary mapping image filenames to their captions."""
    image_captions = {}

    with open(captions_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                # Split on first space to separate image#id from caption
                parts = line.split(' ', 1)
                if len(parts) == 2:
                    img_id, caption = parts
                    # Extract image filename (before the #)
                    img_filename = img_id.split('#')[0]

                    # Store captions in a list for each image
                    if img_filename not in image_captions:
                        image_captions[img_filename] = []
                    image_captions[img_filename].append(caption)

    return image_captions

# Load the dataset
captions_dict = load_captions(captions_file)
image_paths = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])

# Create dataset as list of (image_path, captions) tuples
dataset = []
for img_filename in image_paths:
    img_path = os.path.join(image_dir, img_filename)
    captions = captions_dict.get(img_filename, [])
    if captions:  # Only include images that have captions
        dataset.append({
            'image_path': img_path,
            'image_filename': img_filename,
            'captions': captions
        })

print(f"Loaded {len(dataset)} images with captions")
print(f"Example entry:")
print(f"  Image: {dataset[0]['image_filename']}")
print(f"  Number of captions: {len(dataset[0]['captions'])}")
print(f"  First caption: {dataset[0]['captions'][0]}")

Successfully extracted uic.zip to data/uic
Loaded 3176 images with captions
Example entry:
  Image: uic_img_1.jpg
  Number of captions: 5
  First caption: A dark brown turtle paddles through the water with its limbs .


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from PIL import Image
import random

# Create PyTorch Dataset class
class UICDataset(Dataset):
    def __init__(self, data, processor):
        self.data = data
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = Image.open(item['image_path']).convert('RGB')
        # Use the first caption for training
        caption = item['captions'][0]

        # Process image and text
        encoding = self.processor(
            images=image,
            text=caption,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        # Remove batch dimension
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}

        # Set labels for language modeling
        encoding["labels"] = encoding["input_ids"].clone()

        return encoding

# Split dataset into train and validation (90/10 split)
random.seed(42)
random.shuffle(dataset)
split_idx = int(0.9 * len(dataset))
train_dataset = dataset[:split_idx]
eval_dataset = dataset[split_idx:]

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")

# Create dataset objects
train_data = UICDataset(train_dataset, processor)
eval_data = UICDataset(eval_dataset, processor)

# Configure LoRA for PEFT
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,  # LoRA scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Apply PEFT to model
peft_model = get_peft_model(model, lora_config)
peft_model = peft_model.to(device)
peft_model.print_trainable_parameters()
print(f"PEFT model on device: {next(peft_model.parameters()).device}")

# Training arguments
training_args = TrainingArguments(
    output_dir="./blip2-uic-peft",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-4,
    warmup_steps=100,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=False,
    remove_unused_columns=False,
)

# Create trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
)

# Start training
print("Starting PEFT fine-tuning...")
trainer.train()

# Save the fine-tuned model
peft_model.save_pretrained("./blip2-uic-finetuned")

processor.save_pretrained("./blip2-uic-finetuned")
print("Fine-tuned model saved to ./blip2-uic-finetuned")

Training samples: 2858
Validation samples: 318
trainable params: 5,242,880 || all params: 3,750,004,736 || trainable%: 0.1398
PEFT model on device: cuda:0
Starting PEFT fine-tuning...


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
200,0.684366,0.160552


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [ ]:
# Test the fine-tuned model on a sample image
from peft import PeftModel
from PIL import Image

# Load a test image
test_sample = eval_dataset[0]
test_image = Image.open(test_sample['image_path']).convert('RGB')

# Prepare image for inference
inputs = processor(images=test_image, return_tensors="pt")

# Move inputs to the same device as model
device = next(peft_model.parameters()).device
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate caption
with torch.no_grad():
    generated_ids = peft_model.generate(**inputs, max_length=128)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Test Image:", test_sample['image_filename'])
print("Generated Caption:", generated_caption)
print("\nGround Truth Captions:")
for i, caption in enumerate(test_sample['captions'], 1):
    print(f"  {i}. {caption}")

In [ ]:
import os
import shutil
import json
from datetime import datetime
import tarfile

# Define build directory
BUILD_DIR = "./build"
ARTIFACT_NAME = "blip2-uic-finetuned-artifact"
ARTIFACT_PATH = os.path.join(BUILD_DIR, ARTIFACT_NAME)

# Create build directory structure
os.makedirs(ARTIFACT_PATH, exist_ok=True)

print(f"Creating model artifact in {ARTIFACT_PATH}...")

# Copy model files to artifact directory
model_source = "./blip2-uic-finetuned"
model_dest = os.path.join(ARTIFACT_PATH, "model")

if os.path.exists(model_source):
    shutil.copytree(model_source, model_dest, dirs_exist_ok=True)
    print(f"✓ Copied model files to artifact")
else:
    print(f"⚠ Warning: Model directory {model_source} not found")

# Create metadata file
metadata = {
    "model_name": "BLIP-2 UIC Fine-tuned",
    "base_model": "Salesforce/blip2-opt-2.7b",
    "dataset": "UIC (Underwater Image Captioning)",
    "training_method": "PEFT with LoRA",
    "lora_config": {
        "r": 16,
        "lora_alpha": 32,
        "target_modules": ["q_proj", "v_proj"],
        "lora_dropout": 0.05,
        "task_type": "SEQ_2_SEQ_LM"
    },
    "training_params": {
        "epochs": 3,
        "batch_size": 4,
        "gradient_accumulation_steps": 4,
        "learning_rate": 5e-4,
        "warmup_steps": 100
    },
    "created_at": datetime.now().isoformat(),
    "framework_versions": {
        "transformers": __import__('transformers').__version__,
        "peft": __import__('peft').__version__,
        "torch": torch.__version__
    }
}

metadata_path = os.path.join(ARTIFACT_PATH, "metadata.json")
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Created metadata file")

# Create README for the artifact
readme_content = f"""# BLIP-2 UIC Fine-tuned Model Artifact

## Model Information
- **Base Model**: {metadata['base_model']}
- **Dataset**: {metadata['dataset']}
- **Training Method**: {metadata['training_method']}
- **Created**: {metadata['created_at']}

## Directory Structure
```
{ARTIFACT_NAME}/
├── model/              # Fine-tuned model weights and config
├── metadata.json       # Training configuration and metadata
└── README.md          # This file
```

## Loading the Model

```python
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from peft import PeftModel

# Load processor
processor = Blip2Processor.from_pretrained("./model")

# Load base model
base_model = Blip2ForConditionalGeneration.from_pretrained(
    "{metadata['base_model']}"
)

# Apply PEFT weights
model = PeftModel.from_pretrained(base_model, "./model")

# Use for inference
# ...
```

## Training Details
- **LoRA Rank**: {metadata['lora_config']['r']}
- **LoRA Alpha**: {metadata['lora_config']['lora_alpha']}
- **Learning Rate**: {metadata['training_params']['learning_rate']}
- **Epochs**: {metadata['training_params']['epochs']}
"""

readme_path = os.path.join(ARTIFACT_PATH, "README.md")
with open(readme_path, 'w') as f:
    f.write(readme_content)
print(f"✓ Created README file")

# Create compressed archive
archive_path = os.path.join(BUILD_DIR, f"{ARTIFACT_NAME}.tar.gz")
with tarfile.open(archive_path, "w:gz") as tar:
    tar.add(ARTIFACT_PATH, arcname=ARTIFACT_NAME)

archive_size_mb = os.path.getsize(archive_path) / (1024 * 1024)
print(f"✓ Created compressed archive: {archive_path}")
print(f"  Archive size: {archive_size_mb:.2f} MB")

print("\n" + "="*60)
print("Model artifact packaging complete!")
print("="*60)
print(f"Artifact directory: {ARTIFACT_PATH}")
print(f"Compressed archive: {archive_path}")
print(f"\nTo deploy this model:")
print(f"1. Extract the archive: tar -xzf {archive_path}")
print(f"2. Load the model using the instructions in README.md")

# Verify artifact contents
print("Artifact Contents:")
print("-" * 60)

for root, dirs, files in os.walk(ARTIFACT_PATH):
    level = root.replace(ARTIFACT_PATH, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        file_path = os.path.join(root, file)
        file_size = os.path.getsize(file_path)
        if file_size < 1024:
            size_str = f"{file_size} B"
        elif file_size < 1024 * 1024:
            size_str = f"{file_size / 1024:.1f} KB"
        else:
            size_str = f"{file_size / (1024 * 1024):.1f} MB"
        print(f'{subindent}{file} ({size_str})')

In [ ]:
import zipfile
import os
from datetime import datetime

# Define what to zip
output_dirs = {
    "./blip2-uic-peft": "Training checkpoints and logs",
    "./blip2-uic-finetuned": "Final fine-tuned model"
}

# Create zip filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"blip2_build_{timestamp}.zip"

print(f"Creating {zip_filename}...")
print("=" * 60)

# Create zip file
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for dir_path, description in output_dirs.items():
        if os.path.exists(dir_path):
            print(f"✓ Adding {dir_path} ({description})")
            
            # Walk through directory and add all files
            for root, dirs, files in os.walk(dir_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '.')
                    zipf.write(file_path, arcname)
        else:
            print(f"⚠ Skipping {dir_path} (not found)")

# Get zip file size
zip_size_mb = os.path.getsize(zip_filename) / (1024 * 1024)

print("=" * 60)
print(f"✅ Created: {zip_filename}")
print(f"📦 Size: {zip_size_mb:.2f} MB")
print("\n" + "=" * 60)
print("To download from Google Colab:")
print("=" * 60)
print(f"from google.colab import files")
print(f"files.download('{zip_filename}')")
print("\nOr find it in the Files panel (📁) on the left sidebar")
print("=" * 60)